#### Import Dependencies , 라이브러리 설치

In [25]:
import mediapipe as mp
import cv2
import os
import pandas as pd

In [26]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

mpHands = mp.solutions.hands
#mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5
hands = mpHands.Hands(False, 2, 0.5, 0.5)

#### 비디오 경로설정
- startPath : 폴더가 시작되는 경로
- file_list : class1, class2, class3 ...
- filepath : startPath/file_list/file => 동영상의 프로젝트기준 상대경로
- ex) startPath
-      ㄴ class1
-      ㄴ class2
-      ㄴ class3

In [28]:
import fileControlModule as fcm

startPath = "./KSL_video_sample"      #동영상 샘플이 저장된 시작 루트폴더
fc = fcm.fileControl(startPath)
categories = os.listdir(startPath) # 분류된 폴더명 저장

filepath_list = [] #파일 경로 저장 리스트, 실질적으로 영상을 열기위한 경로로 사용
class_size_list = [] #각 클래스별 파일(영상) 개수

categorie_files = [len(categories)]
for i in range(len(categories)):
    file_list = fc.makeFileList(i)    #의미로 넘버링된 동영상 폴더 접근
    class_size_list.append(len(file_list)) #각 클래스별 파일 수 저장
    
    for j, file in enumerate(file_list):
        filepath = fc.getfilepath(file)   #동영상 하나하나의 경로 지정
        filepath_list.append(filepath)    #리스트에 동영상 경로 저장
    

# print(class_size_list)
# print(categories)
# print(filepath_list)
# print(flie_list)

['./KSL_video_sample/01/00_01.MP4', './KSL_video_sample/01/01_01.MP4', './KSL_video_sample/01/02_01.MP4', './KSL_video_sample/01/03_01.MP4', './KSL_video_sample/01/04_01.MP4', './KSL_video_sample/01/05_01.MP4', './KSL_video_sample/01/06_01.MP4', './KSL_video_sample/01/07_01.MP4', './KSL_video_sample/01/08_01.MP4', './KSL_video_sample/01/09_01.MP4', './KSL_video_sample/01/10_01.MP4', './KSL_video_sample/01/11_01.MP4', './KSL_video_sample/01/12_01.MP4', './KSL_video_sample/01/13_01.MP4', './KSL_video_sample/01/14_01.MP4', './KSL_video_sample/01/15_01.MP4', './KSL_video_sample/01/16_01.MP4', './KSL_video_sample/01/17_01.MP4', './KSL_video_sample/01/18_01.MP4', './KSL_video_sample/01/19_01.MP4', './KSL_video_sample/03/00_03.MP4', './KSL_video_sample/03/01_03.MP4', './KSL_video_sample/03/02_03.MP4', './KSL_video_sample/03/03_03.MP4', './KSL_video_sample/03/04_03.MP4', './KSL_video_sample/03/05_03.MP4', './KSL_video_sample/03/06_03.MP4', './KSL_video_sample/03/07_03.MP4', './KSL_video_sample

#### 랜드마크 정보
- face_landmarks는 pose, hand,와는 다르게 visibility성분이 없다.


In [23]:
results.face_landmarks

landmark {
  x: 0.5086203217506409
  y: 0.29292693734169006
  z: -0.015357264317572117
}
landmark {
  x: 0.506732165813446
  y: 0.26273566484451294
  z: -0.022216567769646645
}
landmark {
  x: 0.5080865621566772
  y: 0.27442947030067444
  z: -0.013267693109810352
}
landmark {
  x: 0.5033676028251648
  y: 0.23561860620975494
  z: -0.013680477626621723
}
landmark {
  x: 0.5063815712928772
  y: 0.2531625032424927
  z: -0.02287854440510273
}
landmark {
  x: 0.5064898729324341
  y: 0.24204657971858978
  z: -0.020227277651429176
}
landmark {
  x: 0.5071949362754822
  y: 0.2167857140302658
  z: -0.006006704177707434
}
landmark {
  x: 0.4785677194595337
  y: 0.22457051277160645
  z: 0.012930437922477722
}
landmark {
  x: 0.5071415305137634
  y: 0.19398868083953857
  z: -0.0008271167171187699
}
landmark {
  x: 0.5067935585975647
  y: 0.18105942010879517
  z: -0.00027058713021688163
}
landmark {
  x: 0.5056125521659851
  y: 0.13918301463127136
  z: 0.008794195018708706
}
landmark {
  x: 0.508799

In [29]:
# for i in enumerate(filepath_list): 전체 동영상 한꺼번에 처리할 경우 사용
cap = cv2.VideoCapture(filepath_list[0])
framecount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(framecount)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        #RGB로 변환
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        #특징점 이미지에 나타내기
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        #렌더링을 위해서 이미지 다시 BGR로 변환
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)
#         cv2.imwrite()
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

108


### 랜드마크 저장 및 CSV로 파일변환

In [30]:
import csv
import os
import numpy as np

In [31]:
features = len(results.face_landmarks.landmark)+len(results.pose_landmarks.landmark)
features

501

#### csv 만들기


In [32]:
landmarks = []
for val in range(1, features+1):
#     landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
    landmarks += ['x{}'.format(val), 'y{}'.format(val)]

landmarks.append('class')

In [33]:
landmarks[-2]

'y501'

In [34]:
with open('dataset.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [35]:
print(categories)

['01', '03', '04', '07', '09', '10']


In [17]:
input_cat = categories[1]

In [20]:
# for i in enumerate(filepath_list): 전체 동영상 한꺼번에 처리할 경우 사용
cap = cv2.VideoCapture(filepath_list[0])

framecount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(framecount)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        #RGB로 변환
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        #특징점 이미지에 나타내기
        results = holistic.process(image)
#         handresults = hands.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        #렌더링을 위해서 이미지 다시 BGR로 변환
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        try:
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y] for landmark in face]).flatten())

            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y] for landmark in pose]).flatten())

#         left_hand = results.left_hand_landmarks.landmark
#         left_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in left_hand]).flatten())

#         right_hand = results.right_hand_landmarks.landmark
#         right_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in right_hand]).flatten())


            row = face_row + pose_row  
            row.append(input_cat)

            with open('dataset.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        
        
        except:
            pass
        
        cv2.imshow('Raw Webcam Feed', image)
#         cv2.imwrite()
        
#         cv2.destroyAllWindows()
#         cv2.waitKey(1)
#         cv2.waitKey(1)
#         cv2.waitKey(1)
#         cv2.waitKey(1)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

128


#### 랜드마크 정보
<!--<img src="https://i.imgur.com/8bForKY.png">-->
<!--<img src="https://i.imgur.com/AzKNp7A.png">-->
- 데이터 shape = class, feature 0 ~ n개의 column으로
- 

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv('dataset.csv')

In [23]:
df.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y497,x498,y498,x499,y499,x500,y500,x501,y501,class
0,0.515594,0.301150,0.514257,0.271725,0.515178,0.282907,0.511497,0.243691,0.514164,0.262101,...,1.593878,0.580052,1.587713,0.459478,1.642439,0.588039,1.678396,0.479574,1.704145,Face
1,0.515986,0.301387,0.514780,0.270768,0.515641,0.282406,0.511627,0.242878,0.514632,0.261118,...,1.580188,0.608734,1.619945,0.467298,1.631175,0.608260,1.679647,0.462179,1.705625,Face
2,0.515127,0.302423,0.513929,0.272160,0.514857,0.283248,0.511013,0.243773,0.513811,0.262532,...,1.582703,0.610860,1.633027,0.473835,1.636785,0.610125,1.702946,0.462664,1.709355,Face
3,0.515285,0.301937,0.514177,0.271634,0.515011,0.282918,0.511076,0.243604,0.514042,0.262026,...,1.610166,0.612320,1.657255,0.472823,1.656954,0.610811,1.729233,0.466499,1.722614,Face
4,0.515449,0.302624,0.514207,0.272319,0.515142,0.283526,0.511237,0.243925,0.514071,0.262652,...,1.602265,0.612671,1.635861,0.474779,1.647826,0.611213,1.714411,0.466904,1.719442,Face


In [24]:
df[df['class']=='Face']

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y497,x498,y498,x499,y499,x500,y500,x501,y501,class
0,0.515594,0.301150,0.514257,0.271725,0.515178,0.282907,0.511497,0.243691,0.514164,0.262101,...,1.593878,0.580052,1.587713,0.459478,1.642439,0.588039,1.678396,0.479574,1.704145,Face
1,0.515986,0.301387,0.514780,0.270768,0.515641,0.282406,0.511627,0.242878,0.514632,0.261118,...,1.580188,0.608734,1.619945,0.467298,1.631175,0.608260,1.679647,0.462179,1.705625,Face
2,0.515127,0.302423,0.513929,0.272160,0.514857,0.283248,0.511013,0.243773,0.513811,0.262532,...,1.582703,0.610860,1.633027,0.473835,1.636785,0.610125,1.702946,0.462664,1.709355,Face
3,0.515285,0.301937,0.514177,0.271634,0.515011,0.282918,0.511076,0.243604,0.514042,0.262026,...,1.610166,0.612320,1.657255,0.472823,1.656954,0.610811,1.729233,0.466499,1.722614,Face
4,0.515449,0.302624,0.514207,0.272319,0.515142,0.283526,0.511237,0.243925,0.514071,0.262652,...,1.602265,0.612671,1.635861,0.474779,1.647826,0.611213,1.714411,0.466904,1.719442,Face
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0.513711,0.309090,0.511339,0.278973,0.512949,0.289215,0.507803,0.251211,0.510906,0.269756,...,1.670493,0.601643,1.727051,0.455985,1.721065,0.581132,1.780445,0.456017,1.781486,Face
126,0.513825,0.309386,0.511699,0.280141,0.513121,0.290444,0.508071,0.251772,0.511324,0.270730,...,1.670383,0.606169,1.730212,0.453855,1.722017,0.585300,1.780902,0.454433,1.781633,Face
127,0.514870,0.310413,0.512455,0.280990,0.513939,0.291405,0.508491,0.251946,0.512006,0.271391,...,1.670373,0.609855,1.724367,0.455440,1.718328,0.591890,1.779637,0.451323,1.778291,Face
128,0.513992,0.313433,0.511650,0.282531,0.513265,0.292600,0.508038,0.253630,0.511234,0.273120,...,1.669983,0.613214,1.725685,0.462352,1.719269,0.593466,1.781258,0.453115,1.780156,Face


In [28]:
X = df.drop('class', axis = 1) # feature들만 x로 분리
y = df['class'] # target value

In [29]:
X

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,x497,y497,x498,y498,x499,y499,x500,y500,x501,y501
0,0.515594,0.301150,0.514257,0.271725,0.515178,0.282907,0.511497,0.243691,0.514164,0.262101,...,0.462559,1.593878,0.580052,1.587713,0.459478,1.642439,0.588039,1.678396,0.479574,1.704145
1,0.515986,0.301387,0.514780,0.270768,0.515641,0.282406,0.511627,0.242878,0.514632,0.261118,...,0.460195,1.580188,0.608734,1.619945,0.467298,1.631175,0.608260,1.679647,0.462179,1.705625
2,0.515127,0.302423,0.513929,0.272160,0.514857,0.283248,0.511013,0.243773,0.513811,0.262532,...,0.469848,1.582703,0.610860,1.633027,0.473835,1.636785,0.610125,1.702946,0.462664,1.709355
3,0.515285,0.301937,0.514177,0.271634,0.515011,0.282918,0.511076,0.243604,0.514042,0.262026,...,0.472045,1.610166,0.612320,1.657255,0.472823,1.656954,0.610811,1.729233,0.466499,1.722614
4,0.515449,0.302624,0.514207,0.272319,0.515142,0.283526,0.511237,0.243925,0.514071,0.262652,...,0.471905,1.602265,0.612671,1.635861,0.474779,1.647826,0.611213,1.714411,0.466904,1.719442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,0.482296,0.531612,0.482465,0.502657,0.482518,0.514993,0.479490,0.480530,0.482534,0.494329,...,0.434624,1.769399,0.559853,1.830921,0.436108,1.820519,0.529447,1.865133,0.433438,1.864522
411,0.482491,0.534621,0.482797,0.505501,0.482719,0.517639,0.479607,0.483438,0.482842,0.497272,...,0.430656,1.773449,0.561380,1.837587,0.434097,1.828325,0.527148,1.870741,0.422536,1.873791
412,0.482302,0.534373,0.482692,0.506140,0.482655,0.518216,0.479571,0.483840,0.482752,0.497820,...,0.428987,1.788002,0.559037,1.845699,0.430479,1.841323,0.528771,1.881279,0.425316,1.888610
413,0.482311,0.534325,0.482586,0.505820,0.482592,0.517875,0.479545,0.483352,0.482658,0.497484,...,0.429839,1.796842,0.557723,1.851370,0.430268,1.847411,0.532702,1.886046,0.429534,1.891785


In [25]:
y

0      Face
1      Face
2      Face
3      Face
4      Face
       ... 
410    Meet
411    Meet
412    Meet
413    Meet
414    Meet
Name: class, Length: 415, dtype: object

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

In [32]:
X_train

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,x497,y497,x498,y498,x499,y499,x500,y500,x501,y501
75,0.512617,0.285082,0.510280,0.254705,0.511860,0.266337,0.507020,0.228244,0.509869,0.245289,...,0.475673,1.641420,0.604388,1.691686,0.483156,1.696153,0.603892,1.753684,0.475741,1.775694
14,0.514517,0.298365,0.513433,0.268484,0.514364,0.279646,0.510529,0.240567,0.513314,0.258931,...,0.464439,1.630801,0.606182,1.670223,0.459255,1.677237,0.602008,1.739226,0.482482,1.737513
394,0.486666,0.555791,0.486110,0.527049,0.486465,0.538099,0.482613,0.502094,0.486040,0.518337,...,0.419562,1.715429,0.552409,1.758603,0.418197,1.758032,0.532402,1.808970,0.426476,1.817318
47,0.514115,0.313029,0.512612,0.283951,0.513957,0.294185,0.509709,0.254868,0.512341,0.274362,...,0.463342,1.629514,0.618533,1.681612,0.465041,1.677821,0.607955,1.740822,0.466478,1.738047
46,0.514208,0.314200,0.512569,0.285318,0.513868,0.295410,0.509337,0.256064,0.512274,0.275721,...,0.463580,1.627390,0.615212,1.680127,0.464445,1.675653,0.602108,1.738718,0.467310,1.736497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,0.516838,0.460134,0.515830,0.429896,0.516289,0.439639,0.510336,0.398437,0.515488,0.419791,...,0.466589,1.918703,0.628758,1.966518,0.471403,1.972975,0.611184,2.018509,0.468111,2.034094
131,0.516357,0.481735,0.515181,0.459829,0.515962,0.466476,0.511576,0.431950,0.514889,0.451373,...,0.458366,1.816104,0.587943,1.870805,0.458886,1.868182,0.560403,1.912997,0.471536,1.926247
249,0.519777,0.455954,0.518261,0.427448,0.518797,0.436721,0.512742,0.396638,0.517849,0.417478,...,0.456374,1.905603,0.627468,1.953420,0.457985,1.962434,0.621944,2.002343,0.468536,2.021750
152,0.528897,0.455685,0.528512,0.424802,0.528296,0.435016,0.522828,0.393778,0.528259,0.414620,...,0.458034,1.988301,0.635118,2.038903,0.460643,2.054916,0.619311,2.084314,0.467104,2.109526


In [33]:
X_test

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,x497,y497,x498,y498,x499,y499,x500,y500,x501,y501
352,0.488262,0.554860,0.489062,0.528057,0.488785,0.538318,0.485833,0.504067,0.489192,0.519761,...,0.409203,1.753567,0.544313,1.798476,0.406743,1.799204,0.527823,1.853333,0.419809,1.858779
130,0.515249,0.482732,0.513996,0.460557,0.514880,0.467279,0.510552,0.432705,0.513715,0.452081,...,0.457420,1.812515,0.587419,1.868749,0.457789,1.864486,0.560000,1.912578,0.470903,1.922530
356,0.488504,0.559987,0.489077,0.534012,0.488879,0.543807,0.485909,0.509450,0.489205,0.525672,...,0.412771,1.752862,0.542268,1.799881,0.409639,1.798791,0.526723,1.854009,0.424326,1.856327
103,0.514068,0.469291,0.513048,0.442018,0.513653,0.450565,0.509773,0.415286,0.512958,0.433513,...,0.467716,1.749504,0.599848,1.799913,0.468763,1.798836,0.572251,1.837113,0.479556,1.855754
310,0.520851,0.455830,0.519267,0.427661,0.519814,0.436820,0.513744,0.397099,0.518851,0.417784,...,0.453054,1.877537,0.624001,1.926298,0.455147,1.931530,0.616444,1.976864,0.462169,1.993909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,0.510624,0.282709,0.508485,0.252040,0.509911,0.264138,0.504982,0.225933,0.508080,0.242604,...,0.468848,1.637688,0.621098,1.675455,0.466661,1.685571,0.614174,1.751256,0.478016,1.748645
251,0.518705,0.456885,0.517261,0.427417,0.517838,0.437002,0.511820,0.396290,0.516874,0.417332,...,0.455459,1.895975,0.623927,1.946287,0.457551,1.950769,0.615235,1.994484,0.467185,2.010761
227,0.526124,0.453613,0.524752,0.424499,0.525086,0.434088,0.518994,0.394355,0.524330,0.414609,...,0.453572,1.893339,0.638355,1.967367,0.454248,1.953037,0.625093,2.013764,0.465467,2.012645
269,0.519258,0.473046,0.518952,0.442441,0.519094,0.452123,0.513700,0.411036,0.518699,0.432444,...,0.426678,1.992396,0.605259,2.040798,0.422976,2.048128,0.587960,2.099366,0.442537,2.119037


## to-do list
- 동영상 파일 카테고리별로 묶고 묶인 것들 cv2로 돌려서 값 뽑아내기
- hand는 별도 처리를 해야 입력값을 받을 수 있기 때문에 이부분 받을 방법 생각해보기
- csv 파일로 데이터 입력할 때 한 영상에서 각각 프레임이 달라서 row개수가 다르게 들어가기 때문에 frame 수 맞춰주고 데이터로 입력
- 모델 적용하기
    - 1D Conv로 feature 줄이고 LSTM에 입력
- 실시간 영상으로 값 저장시키는 작업